In [ ]:
#this notebook is dedicated to training a yolo detection model using the ultralytics library and eventually quantization
# reference vid: https://www.youtube.com/watch?v=r0RspiLG260

In [ ]:
!nvidia-smi

Wed Dec 10 14:42:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!cp /content/drive/MyDrive/surfrider/data_ultralytics.zip /content
!unzip -q /content/data_ultralytics.zip -d /content/data

Mounted at /content/drive/


In [4]:
import os
import shutil
import random

def split_dataset(source_dir, train_dir, val_dir, split_ratio=0.8, seed=42):
    random.seed(seed)

    images_path = os.path.join(source_dir, 'images')
    labels_path = os.path.join(source_dir, 'labels')

    # Get list of all image files (assume .jpg or .png)
    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]
    random.shuffle(image_files)

    # Split indices
    split_idx = int(len(image_files) * split_ratio)
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]

    # Ensure directories exist
    os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

    def copy_files(file_list, dest_dir):
        for img_file in file_list:
            # Copy image
            shutil.copy(os.path.join(images_path, img_file), os.path.join(dest_dir, 'images', img_file))

            # Copy corresponding label (same name, but .txt)
            label_file = os.path.splitext(img_file)[0] + '.txt'
            shutil.copy(os.path.join(labels_path, label_file), os.path.join(dest_dir, 'labels', label_file))

    copy_files(train_files, train_dir)
    copy_files(val_files, val_dir)

    print(f"Copied {len(train_files)} images to {train_dir} and {len(val_files)} images to {val_dir}")

split_dataset('/content/data/', '/content/data/train', '/content/data/validation')


Copied 3424 images to /content/data/train and 856 images to /content/data/validation


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00


In [5]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 10
names:
- Bottle-shaped
- Can-shaped
- Drum
- Easily namable
- Fishing net - cord
- Insulating material
- Other packaging
- Sheet - tarp - plastic bag - fragment
- Tire
- Unclear


In [ ]:
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=50 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs

image 1/856 /content/data/validation/images/001aaa70-130a-4074-84dc-804b4bb2c69d_jpg.rf.a8f2ff648485e2b59aa69f2f39861c47.jpg: 640x480 2 Sheet - tarp - plastic bag - fragments, 42.1ms
image 2/856 /content/data/validation/images/003bc77c-3562-403e-83d7-a2748124f5ad_jpg.rf.e55b66c85d8f9e1c6f194d2508fac107.jpg: 480x640 1 Other packaging, 2 Sheet - tarp - plastic bag - fragments, 36.7ms
image 3/856 /content/data/validation/images/007141b1-454d-472f-8a6c-23a6f1a5a61f_jpg.rf.2bc0436f6053e3356bd11c7daaac63a2.jpg: 640x480 2 Sheet - tarp - plastic bag - fragments, 6.5ms
image 4/856 /content/data/validation/images/00831499-e198-420e-9e41-608512984dc6_jpg.rf.9964225a83199da44761a816572662f0.jpg: 640x480 1 Sheet - tarp - plastic bag - fragment, 6.0ms
image 5/856 /content/data/validation/images/009801bb-dee9-45b0-9fc6-3a394465bae7_jpg.rf.

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

/content/my_model
  adding: my_model.pt (deflated 9%)
  adding: train/ (stored 0%)
  adding: train/train_batch1.jpg (deflated 2%)
  adding: train/val_batch1_pred.jpg (deflated 4%)
  adding: train/val_batch0_labels.jpg (deflated 5%)
  adding: train/confusion_matrix.png (deflated 18%)
  adding: train/results.png (deflated 6%)
  adding: train/val_batch1_labels.jpg (deflated 4%)
  adding: train/train_batch2.jpg (deflated 1%)
  adding: train/val_batch0_pred.jpg (deflated 6%)
  adding: train/train_batch8562.jpg (deflated 5%)
  adding: train/results.csv (deflated 61%)
  adding: train/train_batch8560.jpg (deflated 6%)
  adding: train/val_batch2_labels.jpg (deflated 4%)
  adding: train/BoxP_curve.png (deflated 5%)
  adding: train/BoxPR_curve.png (deflated 10%)
  adding: train/confusion_matrix_normalized.png (deflated 14%)
  adding: train/BoxR_curve.png (deflated 10%)
  adding: train/train_batch8561.jpg (deflated 6%)
  adding: train/weights/ (stored 0%)
  adding: train/weights/best.pt (deflated 

In [ ]:
!cp /content/my_model.zip /content/drive/MyDrive/surfrider/

# Optimization

run this to load saved model (skip if colab didnt disconnect after the hour of training)

In [11]:
from google.colab import drive
drive.mount('/content/drive/')

!cp /content/drive/MyDrive/surfrider/data_ultralytics.zip /content
!unzip -q /content/data_ultralytics.zip -d /content/data

!cp /content/drive/MyDrive/surfrider/my_model.zip /content
!unzip -q /content/my_model.zip -d /content/my_model

DATA_YAML = "/content/data/data.yaml"
MODEL_DIR = "/content/my_model"
MODEL_PT = f"{MODEL_DIR}/my_model.pt"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
replace /content/data/labels/00090d4b-58fa-4e6b-80ee-3bacc15ca7e6_jpg.rf.51fa95529fe0fff24efddf18566771ac.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y
replace /content/my_model/my_model.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/my_model/train/train_batch1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [12]:
!pip install ultralytics

In [13]:
path_to_classes_txt = '/content/data/classes.txt'
path_to_data_yaml = '/content/data/data.yaml'

def split_dataset(source_dir, train_dir, val_dir, split_ratio=0.8, seed=42):
    random.seed(seed)

    images_path = os.path.join(source_dir, 'images')
    labels_path = os.path.join(source_dir, 'labels')

    # Get list of all image files (assume .jpg or .png)
    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]
    random.shuffle(image_files)

    # Split indices
    split_idx = int(len(image_files) * split_ratio)
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]

    # Ensure directories exist
    os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

    def copy_files(file_list, dest_dir):
        for img_file in file_list:
            # Copy image
            shutil.copy(os.path.join(images_path, img_file), os.path.join(dest_dir, 'images', img_file))

            # Copy corresponding label (same name, but .txt)
            label_file = os.path.splitext(img_file)[0] + '.txt'
            shutil.copy(os.path.join(labels_path, label_file), os.path.join(dest_dir, 'labels', label_file))

    copy_files(train_files, train_dir)
    copy_files(val_files, val_dir)

    print(f"Copied {len(train_files)} images to {train_dir} and {len(val_files)} images to {val_dir}")

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

split_dataset('/content/data/', '/content/data/train', '/content/data/validation')
create_data_yaml(path_to_classes_txt, path_to_data_yaml)

Copied 3424 images to /content/data/train and 856 images to /content/data/validation
Created config file at /content/data/data.yaml


In [14]:
from ultralytics import YOLO

model = YOLO(MODEL_PT)
results = model.val(data=DATA_YAML)

print("baseline:", results.results_dict)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2048.9±1155.4 MB/s, size: 336.4 KB)
val: Scanning /content/data/validation/labels... 856 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 856/856 873.5it/s 1.0s
val: New cache created: /content/data/validation/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 54/54 3.3it/s 16.3s
                   all        856       1406      0.289      0.297      0.243      0.128
         Bottle-shaped        161        220      0.419      0.491      0.485      0.212
            Can-shaped         34         40      0.286      0.325      0.236       0.13
                  Drum         13         14      0.481      0.286      0.292      0.192
        Easily namable         34         39      0.144      0.179       0.11

Pruning

In [ ]:
import torch.nn.utils.prune as prune
import torch.nn as nn

def prune_model(yolo_model, amount=0.3):
    for name, module in yolo_model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=amount)
            prune.remove(module, "weight")
    print(f"Pruned {amount*100}% of Conv2d weights.")
    return yolo_model

model_pruned = YOLO(MODEL_PT)
model_pruned.model = prune_model(model_pruned.model, amount=0.3)

model_pruned.train(
    data=DATA_YAML,
    epochs=5,      # 3-10 is enough for recovery
    imgsz=640,
)

PRUNED_PT = f"{MODEL_DIR}/my_model_pruned.pt"
model_pruned.save(PRUNED_PT)

Pruned 30.0% of Conv2d weights.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/my_model/my_model.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pati

In [ ]:
results_pruned = model_pruned.val(data=DATA_YAML)
print("pruned:", results_pruned.results_dict)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3482.2±1015.1 MB/s, size: 272.9 KB)
val: Scanning /content/data/validation/labels.cache... 856 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 856/856 1.9Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 54/54 3.8it/s 14.1s
                   all        856       1406      0.101    0.00676   0.000604   0.000183
         Bottle-shaped        161        220          0          0   4.19e-05   8.37e-06
            Can-shaped         34         40          0          0          0          0
                  Drum         13         14          0          0          0          0
        Easily namable         34         39    0.00301     0.0256   0.000233   4.84e-05
    Fishing net - cord         22         25 

Quantization

In [15]:
from ultralytics import YOLO
import os

DATA_YAML = "/content/data/data.yaml"
MODEL_DIR = "/content/my_model"
MODEL_PT = f"{MODEL_DIR}/my_model.pt"

# Export with opset 21
exported = YOLO(MODEL_PT).export(
    format='onnx',
    project=MODEL_DIR,         # folder to put export
    name="my_model_quantized", # name of the export
    opset=21                   # <--- force opset 21 for Android compatibility
)

# Paths
ONNX_PATH = os.path.join(MODEL_DIR, "my_model.onnx")
ONNX_Q = os.path.join(MODEL_DIR, "my_model_quantized.onnx")

# Rename if needed
os.rename(ONNX_PATH, ONNX_Q)

#if deploying on a phone use this one
#model.export(format='tflite', int8=True, data='/content/data.yaml')

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/my_model/my_model.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 14, 8400) (5.9 MB)

ONNX: starting export with onnx 1.19.1 opset 21...
ONNX: slimming with onnxslim 0.1.78...
ONNX: export success ✅ 1.2s, saved as '/content/my_model/my_model.onnx' (11.7 MB)

Export complete (1.5s)
Results saved to /content/my_model
Predict:         yolo predict task=detect model=/content/my_model/my_model.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/my_model/my_model.onnx imgsz=640 data=/content/data.yaml  
Visualize:       https://netron.app


In [16]:
from ultralytics import YOLO

# onnx int8 quantization
quant_model = YOLO(ONNX_Q)
results_quant = quant_model.val(data=DATA_YAML)
print("quantized:", results_quant.results_dict)

# TFLite INT8 accuracy
# model_tflite = YOLO('/content/my_model/my_model.tflite')
# results_tflite = model_tflite.val(data='/content/data.yaml')
# print("INT8 TFLITE RESULTS:", results_tflite)


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Loading /content/my_model/my_model_quantized.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2660.4±467.4 MB/s, size: 344.3 KB)
val: Scanning /content/data/validation/labels.cache... 856 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 856/856 1.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 856/856 47.3it/s 18.1s
                   all        856       1406      0.326      0.247      0.229      0.122
         Bottle-shaped        161        220      0.489      0.441      0.468      0.206
            Can-s

pruning + quantization

In [ ]:
%cd my_model
!zip /content/my_model.zip my_model.pt my_model_pruned.pt my_model_quantized.onnx
!zip -r /content/my_model.zip train
%cd /content

!cp /content/my_model.zip /content/drive/MyDrive/surfrider/

/content/my_model
updating: my_model.pt (deflated 9%)
updating: my_model_pruned.pt (deflated 9%)
  adding: my_model_quantized.onnx (deflated 14%)
updating: train/ (stored 0%)
updating: train/train_batch1.jpg (deflated 2%)
updating: train/val_batch1_pred.jpg (deflated 4%)
updating: train/val_batch0_labels.jpg (deflated 5%)
updating: train/confusion_matrix.png (deflated 18%)
updating: train/results.png (deflated 6%)
updating: train/val_batch1_labels.jpg (deflated 4%)
updating: train/train_batch2.jpg (deflated 1%)
updating: train/val_batch0_pred.jpg (deflated 6%)
updating: train/train_batch8562.jpg (deflated 5%)
updating: train/results.csv (deflated 61%)
updating: train/train_batch8560.jpg (deflated 6%)
updating: train/val_batch2_labels.jpg (deflated 4%)
updating: train/BoxP_curve.png (deflated 5%)
updating: train/BoxPR_curve.png (deflated 10%)
updating: train/confusion_matrix_normalized.png (deflated 14%)
updating: train/BoxR_curve.png (deflated 10%)
updating: train/train_batch8561.jpg (